In [1]:
import sys
import os
import time
import numpy as np

In [2]:
import sys
import os
import time
import numpy as np

In [13]:
import pydrake
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import SceneGraph
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.optimization import CalcGridPointsOptions, Toppra
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import TrajectorySource
from pydrake.trajectories import PiecewisePolynomial
from pydrake.all import Variable
from pydrake.all import MultibodyPositionToGeometryPose, ConnectMeshcatVisualizer, Role, Sphere
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere,
    Iris, IrisOptions, MakeIrisObstacles, Variable
)
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)

import pydrake.symbolic as sym

from meshcat import Visualizer

# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

def xreplace(expr, rules):
        if isinstance(expr, float) or isinstance(expr, sym.Variable):
            return expr
        assert isinstance(expr, sym.Expression), expr
        for old, new in rules:
            if expr.EqualTo(old):
                return new
        ctor, old_args = expr.Deconstruct()
        new_args = [xreplace(e, rules) for e in old_args]
        return ctor(*new_args)
    
from pydrake.symbolic import RationalFunction

def generate_rationalize_trig_expr_rules(q_var, t_var):
    rules = []
    denom = 1
    for i in range(t.shape[0]):
        sin_rule = (sym.sin(q[i]), (2*t[i])/(1+t[i]**2))
        cos_rule = (sym.cos(q[i]), (1-t[i]**2)/(1+t[i]**2))
        rules += [sin_rule, cos_rule]
#         denom *= (1+t[i]**2)
    return rules


def rationalize_trig_expr(expr, rules):
    return xreplace(expr, rules)

In [14]:
q = sym.MakeVectorContinuousVariable(3,"q")
prog = MathematicalProgram()
t = prog.NewContinuousVariables(q.shape[0], "t")

x1 = sym.sin(q[0])
y1 = -sym.cos(q[1])
x2 = sym.sin(q[0])+ 1/2*sym.sin(q[1])
y2 = sym.cos(q[0]) + sym.cos(q[1])

In [15]:
sub_rules = generate_rationalize_trig_expr_rules(q,t)
u1 = rationalize_trig_expr(x1, sub_rules)
v1 = rationalize_trig_expr(y1, sub_rules)
u2 = rationalize_trig_expr(x2, sub_rules)
v2 = rationalize_trig_expr(y2, sub_rules)

In [17]:
print(type(v2))

<class 'pydrake.symbolic.Expression'>
